In [1]:
# !pip install catboost
# !pip install lightgbm
# !pip install feather-format

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import random
import gc
import lightgbm as lgb
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import feather
import os
import glob
import csv   
from sklearn.metrics import mean_squared_error

Set seed

In [2]:
seed = 7
random.seed(seed)
np.random.seed(seed)

# Load model

In [3]:
name = 'stack'

m = f'models/lgbm_v_{name}_fold_all.bin'
estimator = pickle.load(open(m,'rb'))

In [4]:
estimator.params

{'objective': 'regression',
 'metric': 'rmse',
 'n_jobs': -1,
 'learning_rate': 0.005,
 'num_leaves': 256,
 'max_depth': -1,
 'tree_learner': 'serial',
 'colsample_bytree': 0.9,
 'subsample': 0.5,
 'max_bin': 50,
 'verbose': 2,
 'boosting_type': 'goss',
 'subsample_freq': 0,
 'seed': 7,
 'top_rate': 0.5,
 'other_rate': 0.4}

# Load test data

In [5]:
%%time
path = '../data'
test = feather.read_dataframe(f"{path}/Prepared_data/test_filled_na_light.feather")
gc.collect()

CPU times: user 13.5 s, sys: 32.7 s, total: 46.1 s
Wall time: 1min 26s


7

In [6]:
test.set_index('row_id', inplace=True)

for version in [18,37,40,41]:
    sample_submission = pd.read_csv(f'../data/submission/submission_v{version}.csv')
    sample_submission.set_index('row_id', inplace=True)
    test[f'pred_v{version}'] = sample_submission['meter_reading']
    test[f'pred_v{version}'] = np.log1p(test[f'pred_v{version}'])
    del sample_submission
    gc.collect()
    
test.reset_index(inplace=True)

In [8]:
test['weekday'] = test['utc_time'].dt.weekday
test['weekday'] = test['weekday'].astype('category')

test['dayofyear'] = test['utc_time'].dt.dayofyear
test['dayofyear'] = test['dayofyear'].astype('category')

### Keep only necessary columns

In [11]:
test = test[estimator.feature_name() + ['row_id']]
gc.collect()

137

# Get results

In [12]:
step = 100000
i = 0
test['pred_stacked'] = np.nan

print('here')
while i < test.shape[0]:
    print(i)
    subset = test.loc[i:(i+step)].copy()
    test.loc[i:(i+step),'pred_stacked'] = estimator.predict(subset[estimator.feature_name()])
    i = i + step
    gc.collect()

/Users/palg166k/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


here
0


/Users/palg166k/anaconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [13]:
results = test[['pred_stacked','row_id']]
results['meter_reading'] = np.exp(results['pred_stacked'])-1
results['meter_reading'] = np.where(results['meter_reading'].lt(0),0,results['meter_reading'])
results.sort_values('row_id',inplace=True)

/Users/palg166k/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/palg166k/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/palg166k/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [14]:
results[['row_id','meter_reading']].reset_index(drop=True).to_feather(f'{path}/submission/submission_{name}.feather')
results[['row_id','meter_reading']].reset_index(drop=True).to_csv(f'{path}/submission/submission_{name}.csv', index=False)